In [1]:
%load_ext autoreload
%autoreload 2
import site, sys, os, time
site.addsitedir('..')
from jax.config import config

config.update("jax_enable_x64", True)

In [ ]:
import numpy as np
import jax.numpy as jnp
from  matplotlib import pyplot as plt
import jax
from jax import random
from src.utils import create_2d_mask
from src.interpolate import *

from skimage.data import shepp_logan_phantom
from skimage.transform import radon, rescale, iradon, iradon_sart
from tqdm import tqdm


In [ ]:
def get_fwd_model(pts, rotate_and_project, args = {}):
    
    @jax.jit
    def fwd_model(amp, theta):
        proj, _, _ = rotate_and_project(amp, theta, pts, **args)
        return proj
    
    fwd_model_vmap = jax.jit(jax.vmap(fwd_model, in_axes = (None, 0)))
    
    return fwd_model, fwd_model_vmap

def get_loss_function(fwd_model_vmap, thetas, data):
    
    @jax.jit
    def loss_func(amp): 
        err = fwd_model_vmap(amp, thetas) - data
        return 1/2 * jnp.sum(jnp.real(jnp.conj(err)*err)) / jnp.prod(jnp.array(err.shape))
    
    grad_loss_func = jax.jit(jax.grad(loss_func))
    
    return loss_func, grad_loss_func

In [ ]:
image = shepp_logan_phantom()
image = rescale(image, scale=0.2, mode='reflect')

plt.imshow(image); plt.colorbar()

In [ ]:
thetas_deg = jnp.linspace(0, 360, max(image.shape)+2)
thetas = thetas_deg * jnp.pi/180

sinogram = radon(image, theta = thetas_deg)
print(sinogram.shape)
plt.imshow(sinogram)

### FBP

In [ ]:
# The filter doesn't make a huge difference

rec_fbp = iradon(sinogram, theta=thetas_deg, filter_name='hann')
err_fbp = jnp.abs(rec_fbp - image)
print(f"Error = {jnp.max(err_fbp)}")

plt.rcParams["figure.figsize"]=12,6

plt.subplot(1,2,1)
plt.imshow(rec_fbp); plt.colorbar()

plt.subplot(1,2,2)
plt.imshow(err_fbp); plt.colorbar()

plt.rcParams["figure.figsize"]=6,6

### SART

In [ ]:
N_iter = 2

rec_sart = np.zeros(image.shape)

for i in tqdm(range(N_iter)):
    rec_sart = iradon_sart(sinogram, theta=thetas_deg, image=rec_sart)

err_sart = jnp.abs(rec_sart - image)
print(f"Error = {jnp.max(err_sart)}")

In [ ]:
plt.rcParams["figure.figsize"]=12,6

plt.subplot(1,2,1)
plt.imshow(rec_sart); plt.colorbar()

plt.subplot(1,2,2)
plt.imshow(err_sart); plt.colorbar()

plt.rcParams["figure.figsize"]=6,6

### GMM

In [ ]:
def gen_gauss_x(amp, centre, x, sig):
    return amp * jnp.exp(-(x-centre)**2/(2*sig**2))

gen_gauss_x_vmap = jax.vmap(gen_gauss_x, in_axes=(0, 0, None,None))

def pts2img(amp, pts, x_grid, sig):
    p_x = gen_gauss_x_vmap(amp, pts[:,0], x_grid, sig)
    p_y = gen_gauss_x_vmap(jnp.ones(amp.shape), pts[:,1], x_grid, sig) 
    return jnp.matmul(p_x.transpose(), p_y).transpose() 

#pts2img_vmap = jax.vmap(pts2img_one, in_axes=0)

def rotate_and_project_gmm(amp, theta, pts, x_grid, sig):
    R = jnp.array([[jnp.cos(theta), -jnp.sin(theta)], [jnp.sin(theta), jnp.cos(theta)]])
    pts = (R @ pts.transpose()).transpose()
    img = pts2img(amp, pts, x_grid, sig)
    proj = jnp.sum(img, axis=0)
    
    return proj, img, pts

In [ ]:
nx = image.shape[0]
x_grid = jnp.fft.fftshift(jnp.fft.fftfreq(nx))
sigma = 0.005
amp = 10 + jnp.array(np.random.randn(nx*nx))

rotate_and_project_args = {"x_grid" : x_grid, "sig" : sigma }

In [ ]:
X, Y = jnp.meshgrid(x_grid, x_grid)
pts = jnp.array([X.ravel(), Y.ravel()]).transpose()

In [ ]:
grid_obj = np.array([x_grid[1]-x_grid[0], nx])
mask = jnp.fft.fftshift(create_2d_mask(grid_obj, (0,0), 0.45))

In [ ]:
img = pts2img(amp, pts, x_grid, sigma)

In [ ]:
plt.imshow(img*mask); plt.colorbar()

In [ ]:
theta = jnp.pi/4
proj, img_rot, pts_rot = rotate_and_project_gmm(amp, theta, pts, x_grid, sigma)
plt.imshow(mask*img_rot); plt.colorbar()

In [ ]:
fwd_model, fwd_model_vmap = get_fwd_model(pts, rotate_and_project_gmm, rotate_and_project_args)

N = sinogram.shape[1]
sig_noise = 1000

#thetas = jnp.linspace(0., 2*jnp.pi, N)

data = sinogram.transpose() #fwd_model_vmap(amp, thetas) 
data_noisy = data + np.random.randn(N, nx) * sig_noise

SNR = jnp.sum(data**2)/jnp.sum(data_noisy**2)
print(f"max(data) = {jnp.max(data)}")
print(f"SNR = {SNR}")

In [ ]:
plt.imshow(data)

In [ ]:
loss_func, grad_loss_func = get_loss_function(fwd_model_vmap, thetas, data)

In [ ]:
from src.algorithm import conjugate_gradient

In [ ]:
zero = jnp.zeros(nx**2)
Ab = - grad_loss_func(zero)
AA = lambda v : grad_loss_func(v) + Ab

x0 = jnp.zeros(amp.shape)

x0_cg = conjugate_gradient(AA, Ab, x0, 10, verbose=True)

In [ ]:
N_iters = 200
alpha = 2

x0 = jnp.zeros(amp.shape)

for i in range(N_iters):
    grad = grad_loss_func(x0)
    if jnp.mod(i, 10) == 0:
        #err = jnp.abs(amp-x0).reshape([nx,nx]) * mask
        err = jnp.abs(grad)
        
        print(jnp.sqrt(jnp.sum(err**2)))
        
    x0 = x0 - alpha * grad
    
rec_gmm = pts2img(x0, pts, x_grid, sigma)
err_gmm = jnp.abs(rec_gmm - image)
print(f"Error = {jnp.max(err_gmm)}")

In [ ]:
plt.rcParams["figure.figsize"]=12,6

plt.subplot(1,2,1)
plt.imshow(rec_gmm); plt.colorbar()

plt.subplot(1,2,2)
plt.imshow(err_gmm); plt.colorbar()

plt.rcParams["figure.figsize"]=6,6

### Fourier

In [ ]:
img_f = jnp.fft.fft2(jnp.fft.ifftshift(image)) #*jnp.fft.fftshift(mask)
#img_f = img_f.at[0,0].set(0) #

plt.imshow(jnp.fft.fftshift(jnp.abs(img_f))); plt.colorbar() 

In [ ]:
def rotate_and_project_f(img_f, theta, pts):
    """pts are the Fourier grid points, img_f is the image in Fourier, flattened"""
    
    nx = img_f.shape[0]

    R = jnp.array([[jnp.cos(theta), -jnp.sin(theta)], [jnp.sin(theta), jnp.cos(theta)]])
    pts_rot = R @ pts.transpose()
    pts_rot = jnp.vstack([pts_rot, jnp.zeros((1,nx*nx))])
    x_grid = np.array([1.0, nx])
    
    idx0 = nx*int(nx/2)
    vol = jnp.zeros([nx,nx,nx], dtype=jnp.complex128)

    vol = vol.at[:,:,int(nx/2)].set(jnp.fft.fftshift(img_f))
    vol = jnp.fft.ifftshift(vol)
    img = interpolate(pts_rot, x_grid, vol, 'tri')
    
    #print(pts[idx0:idx0+nx,:])

    proj = img[idx0:idx0+nx]
    
    return proj, img, pts

In [ ]:
# Get the Fourier grid
x_grid_f = jnp.arange(-nx/2, nx/2)
X, Y = jnp.meshgrid(x_grid_f, x_grid_f)
pts = jnp.array([X.ravel(), Y.ravel()]).transpose()

In [ ]:
theta = jnp.pi/4
proj, img_rot, pts_rot = rotate_and_project_f(img_f, theta, pts)

plt.imshow((jnp.abs(img_rot.reshape([nx,nx])))); plt.colorbar()

In [ ]:
plt.imshow(jnp.fft.fftshift(jnp.abs(img_f))); plt.colorbar()

In [ ]:
img_rot_r = jnp.real(jnp.fft.fftshift(jnp.fft.ifftn(jnp.fft.ifftshift(img_rot.reshape([nx,nx])))))
plt.imshow(img_rot_r); plt.colorbar()

In [ ]:
fwd_model, fwd_model_vmap = get_fwd_model(pts, rotate_and_project_f)

N = sinogram.shape[1]
sig_noise = 10000

#thetas = 2 * jnp.pi * jnp.array(np.random.rand(N)) 

data = sinogram.transpose() #fwd_model_vmap(img_f, thetas) 
data_noisy = data + (np.random.randn(N, nx) + 1j*np.random.randn(N, nx))* sig_noise

SNR = jnp.real(jnp.sum(jnp.conj(data)*data)/jnp.sum(jnp.conj(data_noisy)*data_noisy))
print(f"max(data) = {jnp.max(jnp.abs(data))}")
print(f"SNR = {SNR}")

In [ ]:
loss_func, grad_loss_func = get_loss_function(fwd_model_vmap, thetas, data)

In [ ]:
N_iters = 1000
alpha = 1e-1

x0 = jnp.zeros(img_f.shape, dtype = jnp.complex128)
#x0 = img_f + 1

for i in range(N_iters):
    if jnp.mod(i, 100) == 0:
        err = jnp.abs(img_f-x0).reshape([nx,nx]) * mask

        print(jnp.sqrt(jnp.sum(err**2)))
        
    x0 = x0 - alpha * grad_loss_func(x0)

In [ ]:
plt.imshow(jnp.fft.fftshift(jnp.abs(x0))); plt.colorbar()

In [ ]:
plt.imshow(jnp.fft.fftshift(jnp.real(jnp.fft.ifftn(x0)))); plt.colorbar()